## Load Questions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/RAG_Optimizer/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/RAG_Optimizer


In [2]:
!pip install -U --ignore-installed \
  torch \
  transformers \
  sentence-transformers \
  langchain \
  langchain-community \
  faiss-cpu \
  rank-bm25 \
  pandas \
  numpy \
  scikit-learn \
  spacy \
  tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 749.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 152.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import pandas as pd
import re
from itertools import product
import spacy
from collections import Counter
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from ..ingest import ingest_data
from ..run_rag import run_rag_pipeline

In [3]:
samples = pd.read_csv("questions.csv")

## Core Combinations

In [4]:
RETRIEVERS = ["faiss", "bm25", "hybrid"]
CHUNK_SIZE = [400, 800]
ANS_MODEL = ["qa", "llm"]
K = [3, 5]
RERANKER = [True, False]

In [5]:
configs = []
config_id = 0

from itertools import product

for ret, chk_size, model, k, rerank in product(
    RETRIEVERS,
    CHUNK_SIZE,
    ANS_MODEL,
    K,
    RERANKER
):
    # qa models doesn't perform well on large k values
    if model == "qa" and k > 5:
        continue
    if ret == "bm25" and rerank:
        continue
    if rerank and k < 5:
        continue
    if model == "qa" and ret == "hybrid" and rerank:
        continue

    configs.append({
        "config_id" : config_id,
        "Retriever" : ret,
        "Chunk_size" : chk_size,
        "Chunk_overlap" : 100,
        "k" : k,
        "Answer_model" : model,
        "Reranker" : rerank
    })
    config_id += 1

In [6]:
len(configs)

30

## Functions for question's feature extraction

In [7]:
def normalize(s):
    s = re.sub(r"\b(a|an|the)\b", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    s = re.sub(r"\s+", " ", s)

    return s.lower().strip()

In [8]:
def question_type(question):
    q = question.lower()

    if any(w in q for w in ["explain", "describe", "why", "how"]):
        return "explanatory"

    if any(w in q for w in ["list", "all", "authors", "members"]):
        return "factoid_list"

    if q.startswith(("who", "when", "what")):
        return "factoid_single"

    return "other"

In [9]:
nlp = spacy.load("en_core_web_sm")

def entity_count(question):
    doc = nlp(question)

    return len(doc.ents)

In [10]:
def get_features(question):
    tokens = normalize(question).split()

    return {
        "tokens_length" : len(tokens),
        "isdigit" : any(c.isdigit() for c in question),
        "num_entities" : entity_count(question),
        "question_type" : question_type(question)
    }

## Dataset Generation

In [11]:
# f1 score for normal qa models
def f1(truth, pred):
    pred_tokens = normalize(pred).split()
    truth_tokens = normalize(truth).split()

    commons = Counter(pred_tokens) & Counter(truth_tokens)
    num_commons = sum(commons.values())

    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return pred_tokens == truth_tokens

    if num_commons == 0:
        return 0.0

    precision = num_commons / len(pred_tokens)
    recall = num_commons / len(truth_tokens)

    return 2 * precision * recall / (len(pred_tokens) + len(truth_tokens))

In [12]:
model = SentenceTransformer("all-MiniLM-L6-v2")

# semantic similarity for LLMs
def sementic_similarity(truth, pred):
    truth_embed = model.encode(truth, convert_to_tensor = True)
    pred_embed = model.encode(pred, convert_to_tensor = True)

    return util.cos_sim(truth_embed, pred_embed).item()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
reranker = CrossEncoder(
            "cross-encoder/ms-marco-MiniLM-L6-v2"
        )

embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

pipe = pipeline("text2text-generation", model="google/flan-t5-base", max_new_tokens = 512)
llm = HuggingFacePipeline(pipeline = pipe)

qa_model = pipeline("question-answering", model = "deepset/roberta-base-squad2")

/tmp/ipython-input-2788187893.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
Device set to use cuda:0
/tmp/ipython-input-2788187893.py:8: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = pipe)
Device set to use cuda:0


In [14]:
metadata = None

In [15]:
rows = []
i = 0

for cfg in configs:
    for _, row in samples.iterrows():
        question = row["question"]
        truth = row["answer"]
        context = row["context"]
        q_features = get_features(question)

        if metadata != [cfg["Retriever"], cfg["Chunk_size"], cfg["Chunk_overlap"], context]:

            metadata = [cfg["Retriever"], cfg["Chunk_size"], cfg["Chunk_overlap"], context]

            ingest_data(
                context = context,
                chunk_size = cfg["Chunk_size"],
                chunk_overlap = cfg["Chunk_overlap"],
                retriever = cfg["Retriever"]
            )

        pred = run_rag_pipeline(
            retriever = cfg["Retriever"],
            k = cfg["k"],
            answer_model = qa_model if cfg["Answer_model"] == "qa" else llm,
            answer_type = cfg["Answer_model"],
            reranker = reranker,
            need_rerank = cfg["Reranker"],
            embedding = embedding,
            question = question
        )

        f1_score = f1(truth, pred)
        sementic_similarity_score = sementic_similarity(pred, truth)

        rows.append({
            "question" : question,
            "truth" : truth,
            "prediction" : pred,
            "f1" : f1_score,
            "sementic_similarity" : sementic_similarity_score,
            **q_features,
            **cfg
        })
        print(i)
        i += 1


0
1
2
3
4
5
6
7


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


Token indices sequence length is longer than the specified maximum sequence length for this model (550 > 512). Running this sequence through the model will result in indexing errors


225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474


In [ ]:
pd.DataFrame(rows).to_csv("dataset.csv")

In [17]:
rows[:10]

[{'question': 'When did Beyonce start becoming popular?',
  'truth': 'in the late 1990s',
  'prediction': 'late 1990s',
  'f1': 0.26666666666666666,
  'sementic_similarity': 0.9371858835220337,
  'tokens_length': 6,
  'isdigit': False,
  'num_entities': 1,
  'question_type': 'factoid_single',
  'config_id': 0,
  'Retriever': 'faiss',
  'Chunk_size': 400,
  'Chunk_overlap': 100,
  'k': 3,
  'Answer_model': 'qa',
  'Reranker': False},
 {'question': 'What areas did Beyonce compete in when she was growing up?',
  'truth': 'singing and dancing',
  'prediction': 'singing and dancing',
  'f1': 0.3333333333333333,
  'sementic_similarity': 1.0,
  'tokens_length': 11,
  'isdigit': False,
  'num_entities': 0,
  'question_type': 'factoid_single',
  'config_id': 0,
  'Retriever': 'faiss',
  'Chunk_size': 400,
  'Chunk_overlap': 100,
  'k': 3,
  'Answer_model': 'qa',
  'Reranker': False},
 {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
  'truth': '2003',
  'predicti